In [1]:
# 예제 1: N-그램(2-gram) 언어 모델 - 한글 문장 확률 계산
from collections import defaultdict

# 예시 텍스트(작은 가사 일부)
text = "내일은 더 행복할 거야 오늘보다 더 많이 웃을 거야"

# 2-gram(바이그램) 모델 생성
words = text.split()
bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]

# 바이그램 확률 딕셔너리 만들기
bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)
for w1, w2 in bigrams:
    bigram_counts[(w1, w2)] += 1
    unigram_counts[w1] += 1

# 조건부 확률 계산 함수
def bigram_prob(w1, w2):
    return bigram_counts[(w1, w2)] / unigram_counts[w1]

# 실제로 확률 계산해보기
print(f"('더', '행복할') 확률:", bigram_prob('더', '행복할'))
print(f"('더', '많이') 확률:", bigram_prob('더', '많이'))


('더', '행복할') 확률: 0.5
('더', '많이') 확률: 0.5


In [2]:
from collections import defaultdict, Counter
import random

# [1] 예시 데이터 (다양한 뉴스/가사/명언 등 복수 문장)
corpus = [
    "오늘도 밝은 태양이 떠오른다",
    "희망찬 아침이 시작된다",
    "슬픔은 이겨내야 한다",
    "행복은 소소한 곳에 있다",
    "성공은 노력의 결과이다",
    "모든 순간이 기회다",
    "행복을 찾는 여행을 시작해봐",
    "고난은 성장의 밑거름이 된다",
    "오늘은 내일의 꿈을 준비하는 날이다",
    "삶은 도전과 응전의 연속이다"
]

# [2] 2-gram 카운트/확률 테이블 구축 (Laplace smoothing)
bigram_counts = defaultdict(lambda: defaultdict(int))
unigram_counts = Counter()
vocab = set()

for sent in corpus:
    tokens = sent.split()
    vocab.update(tokens)
    for i in range(len(tokens)-1):
        w1, w2 = tokens[i], tokens[i+1]
        bigram_counts[w1][w2] += 1
        unigram_counts[w1] += 1
    unigram_counts[tokens[-1]] += 1  # 마지막 단어

vocab = list(vocab)
V = len(vocab)  # 단어 개수

def bigram_prob(w1, w2, alpha=1):
    # Laplace smoothing
    return (bigram_counts[w1][w2] + alpha) / (unigram_counts[w1] + alpha*V)

# [3] 문장 확장/생성 함수
def generate_sentence(seed, length=7):
    tokens = seed.split()
    for _ in range(length):
        w1 = tokens[-1]
        candidates = [(w2, bigram_prob(w1, w2)) for w2 in vocab]
        # 확률에 따라 단어 샘플링 (softmax 아님, 실제론 분포대로 샘플링)
        candidates = sorted(candidates, key=lambda x: -x[1])
        w2 = random.choices([w for w, p in candidates], [p for w, p in candidates])[0]
        tokens.append(w2)
        if w2.endswith('다') or w2.endswith('다.'):  # 간단한 문장 마침 처리
            break
    return ' '.join(tokens)

# [4] 다음 단어 예측 함수 (확률 순위 TOP 5)
def predict_next(word, topk=5):
    cands = [(w2, bigram_prob(word, w2)) for w2 in vocab]
    cands = sorted(cands, key=lambda x: -x[1])[:topk]
    print(f"'{word}' 다음에 올 가능성 높은 단어:")
    for w2, p in cands:
        print(f"  {w2}  (P={p:.3f})")

# [5] 테스트
print("== 2-gram 기반 문장 생성 예시 ==")
print(generate_sentence("행복은"))

print("\n== 다음 단어 예측 예시 ==")
predict_next("오늘도")
predict_next("성공은")
predict_next("고난은")

# [6] 2-gram 확률 확인 예시
print("\n== 특정 bigram 확률 ==")
w1, w2 = "행복은", "소소한"
print(f"P({w2}|{w1}) =", round(bigram_prob(w1, w2), 3))
w1, w2 = "모든", "순간이"
print(f"P({w2}|{w1}) =", round(bigram_prob(w1, w2), 3))
w1, w2 = "고난은", "성장의"
print(f"P({w2}|{w1}) =", round(bigram_prob(w1, w2), 3))


== 2-gram 기반 문장 생성 예시 ==
행복은 있다

== 다음 단어 예측 예시 ==
'오늘도' 다음에 올 가능성 높은 단어:
  밝은  (P=0.053)
  떠오른다  (P=0.026)
  노력의  (P=0.026)
  성공은  (P=0.026)
  한다  (P=0.026)
'성공은' 다음에 올 가능성 높은 단어:
  노력의  (P=0.053)
  떠오른다  (P=0.026)
  성공은  (P=0.026)
  한다  (P=0.026)
  날이다  (P=0.026)
'고난은' 다음에 올 가능성 높은 단어:
  성장의  (P=0.053)
  떠오른다  (P=0.026)
  노력의  (P=0.026)
  성공은  (P=0.026)
  한다  (P=0.026)

== 특정 bigram 확률 ==
P(소소한|행복은) = 0.053
P(순간이|모든) = 0.053
P(성장의|고난은) = 0.053


In [3]:
from collections import defaultdict, Counter
import random

# [1] 복수 문장 데이터 (한글 뉴스/가사/명언 등)
corpus = [
    "지금 이 순간을 소중히 하세요",
    "행복은 마음속에 있어요",
    "내일은 더 나아질 거예요",
    "고마운 마음을 잊지 마세요",
    "친구는 소중한 보물입니다",
    "희망은 포기하지 않는 마음입니다",
    "슬픔은 곧 지나갈 거예요",
    "오늘도 좋은 하루 되세요",
    "사랑은 모든 것을 이겨냅니다",
    "용기는 두려움을 이기는 힘입니다"
]

# [2] 2-gram & 3-gram 테이블 구축
bigram = defaultdict(lambda: defaultdict(int))
trigram = defaultdict(lambda: defaultdict(int))
unigram = Counter()
vocab = set()

for sent in corpus:
    tokens = sent.split()
    vocab.update(tokens)
    for i in range(len(tokens)-1):
        w1, w2 = tokens[i], tokens[i+1]
        bigram[w1][w2] += 1
        unigram[w1] += 1
    for i in range(len(tokens)-2):
        w1, w2, w3 = tokens[i], tokens[i+1], tokens[i+2]
        trigram[(w1, w2)][w3] += 1
    unigram[tokens[-1]] += 1

vocab = list(vocab)
V = len(vocab)

def bigram_prob(w1, w2, alpha=1):
    return (bigram[w1][w2] + alpha) / (unigram[w1] + alpha*V)

def trigram_prob(w1, w2, w3, alpha=1):
    return (trigram[(w1, w2)][w3] + alpha) / (bigram[w1][w2] + alpha*V)

# [3] 문장 자동완성/생성 함수 (N-gram 혼합, 우선 trigram → bigram)
def complete_sentence(seed, max_length=8):
    tokens = seed.split()
    while len(tokens) < max_length:
        if len(tokens) >= 2:
            w1, w2 = tokens[-2], tokens[-1]
            candidates = [(w3, trigram_prob(w1, w2, w3)) for w3 in vocab]
            candidates = sorted(candidates, key=lambda x: -x[1])
            next_word = candidates[0][0] if candidates[0][1] > 0 else None
        else:
            w1 = tokens[-1]
            candidates = [(w2, bigram_prob(w1, w2)) for w2 in vocab]
            candidates = sorted(candidates, key=lambda x: -x[1])
            next_word = candidates[0][0] if candidates[0][1] > 0 else None
        if not next_word:
            break
        tokens.append(next_word)
        if next_word.endswith('다') or next_word.endswith('다.'):
            break
    return ' '.join(tokens)

# [4] 자동완성 추천 함수 (n-gram 기반 다음 단어 추천 TOP 5)
def suggest_next(seed, topk=5):
    tokens = seed.split()
    if len(tokens) >= 2:
        w1, w2 = tokens[-2], tokens[-1]
        cands = [(w3, trigram_prob(w1, w2, w3)) for w3 in vocab]
        cands = sorted(cands, key=lambda x: -x[1])[:topk]
    else:
        w1 = tokens[-1]
        cands = [(w2, bigram_prob(w1, w2)) for w2 in vocab]
        cands = sorted(cands, key=lambda x: -x[1])[:topk]
    print(f"입력: '{seed}' → 추천 단어:")
    for w, p in cands:
        if p > 0:
            print(f"  {w} (확률={p:.3f})")

# [5] 실습 테스트
print("== 문장 자동완성 ==")
print(complete_sentence("행복은"))

print(complete_sentence("오늘도 좋은"))
print(complete_sentence("고마운 마음을"))
print(complete_sentence("친구는"))

print("\n== 다음 단어 추천 ==")
suggest_next("내일은 더")
suggest_next("희망은")
suggest_next("용기는 두려움을")

print("\n== 새로운 문장 자동완성 ==")
start = "희망은 포기하지"
print(f"'{start}' →", complete_sentence(start))


== 문장 자동완성 ==
행복은 마음속에 있어요 보물입니다
오늘도 좋은 하루 되세요 보물입니다
고마운 마음을 잊지 마세요 보물입니다
친구는 소중한 보물입니다

== 다음 단어 추천 ==
입력: '내일은 더' → 추천 단어:
  나아질 (확률=0.051)
  보물입니다 (확률=0.026)
  사랑은 (확률=0.026)
  모든 (확률=0.026)
  마음속에 (확률=0.026)
입력: '희망은' → 추천 단어:
  포기하지 (확률=0.051)
  보물입니다 (확률=0.026)
  나아질 (확률=0.026)
  사랑은 (확률=0.026)
  모든 (확률=0.026)
입력: '용기는 두려움을' → 추천 단어:
  이기는 (확률=0.051)
  보물입니다 (확률=0.026)
  나아질 (확률=0.026)
  사랑은 (확률=0.026)
  모든 (확률=0.026)

== 새로운 문장 자동완성 ==
'희망은 포기하지' → 희망은 포기하지 않는 마음입니다
